In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def add_time_features(df):
    df['OutDate'] = pd.to_datetime(df['OutDate'])
    df['Sales'] = df['Sales'].astype('category')
    df['Customer'] = df['Customer'].astype('category')
    df['Product'] = df['Product'].astype('category')
    df['ProductID'] = df['ProductID'].astype('category')
    df['Year'] = df['OutDate'].dt.year
    df['Month'] = df['OutDate'].dt.month
    df['Day'] = df['OutDate'].dt.day
    df['Weekday'] = df['OutDate'].dt.weekday
    return df

### Data Preparation


In [ ]:
df = pd.read_csv('Sales_Summary.csv')

df = add_time_features(df)

df.head()

In [ ]:
df.dtypes

In [ ]:
# Group by Year and Weekday
grouped_data = df.groupby(['Year', 'Weekday'])['TotalSales'].sum().reset_index()

# Mapping numbers to weekday names
weekday_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday',
               4: 'Friday', 5: 'Saturday', 6: 'Sunday'}

grouped_data['Weekday'] = grouped_data['Weekday'].map(weekday_map)

plt.figure(figsize=(12, 6))

# Get unique years
years = grouped_data['Year'].unique()

for year in years:
    # Filter data for each year
    yearly_data = grouped_data[grouped_data['Year'] == year]

    # Plot
    sns.lineplot(x='Weekday', y='TotalSales', data=yearly_data, label=str(year))

plt.title('Total Sales by Weekday for Each Year')
plt.xlabel('Weekday')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.legend(title='Year')
plt.show()


In [ ]:
grouped_data = df.groupby(['Year', 'Month'])['TotalSales'].sum().reset_index()

plt.figure(figsize=(12, 6))

# Get unique years
years = grouped_data['Year'].unique()

for year in years:
    # Filter data for each year
    yearly_data = grouped_data[grouped_data['Year'] == year]

    # Plot
    sns.lineplot(x='Month', y='TotalSales', data=yearly_data, label=str(year))

plt.title('Total Sales by Month for Each Year')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.legend(title='Year')
plt.show()


In [ ]:
# List of dimensions for which you want to plot summaries
dimensions = ['Year', 'Month', 'Day', 'Weekday', 'Sales']

for dimension in dimensions:
    plt.figure(figsize=(15, 6))
    ax = sns.barplot(x=dimension, y='TotalSales', data=df, estimator=lambda x: sum(x) / 1e6, ci=None)

    # Format y-axis in millions
    ax.set_ylabel('Total Sales (in Millions)')
    ax.set_yticklabels([f'{y:.1f}M' for y in ax.get_yticks()])

    # Add labels on each bar
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.1f}M',  # Format the label in millions
                    (p.get_x() + p.get_width() / 2., p.get_height()),  # Position for the label
                    ha='center', va='center',  # Alignment
                    xytext=(0, 10),  # Distance from the bar
                    textcoords='offset points')

    plt.title(f'Total Sales by {dimension}')
    plt.xticks(rotation=45)  # Rotate labels for better readability if needed
    plt.xlabel(dimension)
    plt.show()

In [ ]:
df['YearMonth'] = df['OutDate'].dt.to_period('M')
sales_categories = df['Sales'].unique()

# Set the size of the plots
plt.figure(figsize=(15, 6))

# Loop through each sales category and plot
for sales_category in sales_categories:
    # Filter data for the current category
    category_data = df[df['Sales'] == sales_category]

    # Group by YearMonth and sum the TotalSales
    grouped_data = category_data.groupby('YearMonth')['TotalSales'].sum().reset_index()

    # Plot
    plt.plot(grouped_data['YearMonth'].dt.to_timestamp(), grouped_data['TotalSales'], label=sales_category)

# Add plot details
plt.title('Total Sales by Sales Category Over Time')
plt.xlabel('Year-Month')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.legend(title='Sales Category')
plt.show()

In [ ]:
df

In [ ]:
total_sales_df = df.groupby('OutDate')['TotalSales'].sum().reset_index()
total_sales_df

In [ ]:
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['OutDate']
  ys = series['TotalSales']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(15, 5.2), layout='constrained')
df_sorted = total_sales_df.sort_values('OutDate', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('OutDate')
_ = plt.ylabel('TotalSales')

In [ ]:
# Aggregate data for forecast by Sales category
sales_category_df = df.groupby(['OutDate', 'Sales'])['TotalSales'].sum().reset_index()
sales_category_df

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5.2), layout='constrained')

# Get unique sales categories
sales_categories = sales_category_df['Sales'].unique()

# Plot each category
for index, category in enumerate(sales_categories):
    category_df = sales_category_df[sales_category_df['Sales'] == category].sort_values('OutDate', ascending=True)
    _plot_series(category_df, category, series_index=index)

# Finalize the plot
sns.despine(fig=fig, ax=ax)
plt.xlabel('OutDate')
plt.ylabel('TotalSales')
plt.legend(title='Sales Category')
plt.show()

### Feature Engineering

In [ ]:


total_sales_df = add_time_features(total_sales_df)
sales_category_df = add_time_features(sales_category_df)
total_sales_df

In [ ]:
sales_category_df

### Train-Test Split

In [ ]:
# Define a split date as a Timestamp object
split_date = pd.Timestamp('2023-01-01')

# For total forecast
train_total = total_sales_df[total_sales_df['OutDate'] < split_date]
test_total = total_sales_df[total_sales_df['OutDate'] >= split_date]

# For Sales category forecast
train_sales_category = sales_category_df[sales_category_df['OutDate'] < split_date]
# train_sales_category = train_data.asfreq('D', fill_value=0)

test_sales_category = sales_category_df[sales_category_df['OutDate'] >= split_date]

train_total.shape, test_total.shape, train_sales_category.shape, test_sales_category.shape

### Model Training

#### a. Total Forecast Model


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Prepare the data
X_train_total = train_total.drop(['TotalSales', 'OutDate'], axis=1)
y_train_total = train_total['TotalSales']

X_test_total = test_total.drop(['TotalSales', 'OutDate'], axis=1)
y_test_total = test_total['TotalSales']

# Train the model
model_total = LinearRegression()
model_total.fit(X_train_total, y_train_total)

In [ ]:
import statsmodels.api as sm

# Ensure 'OutDate' is the index and the data is sorted
# train_total = train_total.set_index('OutDate').sort_index()
# test_total = test_total.set_index('OutDate').sort_index()

# Set the frequency of the datetime index
train_total.index = pd.DatetimeIndex(train_total.index).to_period('D')
test_total.index = pd.DatetimeIndex(test_total.index).to_period('D')

# Define and fit the SARIMA model - example parameters, adjust based on your data
model_total = sm.tsa.statespace.SARIMAX(train_total['TotalSales'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
results_total = model_total.fit()

#### b. Sales Category Forecast Model


In [ ]:
models_by_sales = {}

for sales_category in train_sales_category['Sales'].unique():
    train_data = train_sales_category[train_sales_category['Sales'] == sales_category]
    X_train = train_data.drop(['TotalSales', 'OutDate', 'Sales'], axis=1)
    y_train = train_data['TotalSales']

    model = LinearRegression()
    model.fit(X_train, y_train)
    models_by_sales[sales_category] = model


In [ ]:
models_by_sales = {}

for sales_category in train_sales_category['Sales'].unique():
    train_data = train_sales_category[train_sales_category['Sales'] == sales_category]
    train_data.index = pd.DatetimeIndex(train_data.index).to_period('M')

    # Define and fit the SARIMA model for each category
    model = sm.tsa.statespace.SARIMAX(train_data['TotalSales'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    results = model.fit()
    models_by_sales[sales_category] = results

### Model Evaluation


#### a. Total Forecast Model Evaluation


In [ ]:
y_pred_total = model_total.predict(X_test_total)
mse_total = mean_squared_error(y_test_total, y_pred_total)
print(f'Total Forecast MSE: {mse_total}')

#### b. Sales Category Forecast Model Evaluation


In [ ]:
mse_by_sales = {}

for sales_category, model in models_by_sales.items():
    test_data = test_sales_category[test_sales_category['Sales'] == sales_category]
    X_test = test_data.drop(['TotalSales', 'OutDate', 'Sales'], axis=1)
    y_test = test_data['TotalSales']

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_by_sales[sales_category] = mse

for sales_category, mse in mse_by_sales.items():
    print(f'{sales_category} Forecast MSE: {mse}')


### Forecasting


#### Creating Future Dates

In [ ]:
# Create a date range for 2024, assuming monthly forecasts
future_dates = pd.date_range(start='2023-11-08', end='2024-12-31', freq='MS')
future_dates_df = pd.DataFrame(future_dates, columns=['OutDate'])

# Add additional time features if they were used in the model
future_dates_df['Year'] = future_dates_df['OutDate'].dt.year
future_dates_df['Month'] = future_dates_df['OutDate'].dt.month
future_dates_df['Day'] = future_dates_df['OutDate'].dt.day
future_dates_df['Weekday'] = future_dates_df['OutDate'].dt.weekday

#### a. Total Forecast


In [ ]:
# Drop 'OutDate' as it was not used in the model
X_future_total = future_dates_df.drop(['OutDate'], axis=1)

# Make predictions
future_total_sales_predictions = model_total.predict(X_future_total)

# Combine predictions with dates for better readability
total_forecast = pd.DataFrame({
    'Date': future_dates_df['OutDate'],
    'Predicted Total Sales': future_total_sales_predictions
})

#### b. Sales Category Forecast


In [ ]:
future_sales_predictions = {}

for sales_category in train_sales_category['Sales'].unique():
    # Create a DataFrame for each category
    future_df = future_dates_df.copy()
    future_df['Sales'] = sales_category  # Add the sales category

    # Add any other necessary features or transformations

    # Prepare the data for the model
    X_future = future_df.drop(['OutDate', 'Sales'], axis=1)

    # Make predictions
    model = models_by_sales[sales_category]
    future_sales_predictions[sales_category] = model.predict(X_future)

    # Combine predictions with dates
    future_sales_predictions[sales_category] = pd.DataFrame({
        'Date': future_dates_df['OutDate'],
        'Predicted Sales for ' + sales_category: future_sales_predictions[sales_category]
    })


### Plotting

#### a. Plot actual and forecasted for Total

In [ ]:
# Historical data (up to 2023)
historical_sales_df = total_sales_df
historical_sales_df['OutDate'] = pd.to_datetime(historical_sales_df['OutDate'])
historical_sales_df = historical_sales_df.sort_values('OutDate')

# Forecasted data (for 2024)
total_forecast['Date'] = pd.to_datetime(total_forecast['Date'])

In [ ]:
plt.figure(figsize=(25, 6))

# Plot historical data
plt.plot(historical_sales_df['OutDate'], historical_sales_df['TotalSales'], label='Actual Sales', color='blue')

# Plot forecasted data
plt.plot(total_forecast['Date'], total_forecast['Predicted Total Sales'], label='Forecasted Sales', color='red', linestyle='--')

plt.title('Total Sales: Actual vs Forecasted')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.legend()
plt.grid(True)
plt.show()


#### b. Plot for Actual and Forecated for Sales Category

In [ ]:
# Convert 'OutDate' to datetime and sort
historical_sales_by_category_df = sales_category_df
historical_sales_by_category_df['OutDate'] = pd.to_datetime(historical_sales_by_category_df['OutDate'])
historical_sales_by_category_df = historical_sales_by_category_df.sort_values('OutDate')

In [ ]:
sales_categories = historical_sales_by_category_df['Sales'].unique()

# Set up the plot
plt.figure(figsize=(25, 8))

for category in sales_categories:
    # Filter historical data for the category
    historical_data = historical_sales_by_category_df[historical_sales_by_category_df['Sales'] == category]

    # Filter forecasted data for the category
    forecast_data = future_sales_predictions[category]

    # Plot historical data
    plt.plot(historical_data['OutDate'], historical_data['TotalSales'], label=f'Actual Sales - {category}')

    # Plot forecasted data
    plt.plot(forecast_data['Date'], forecast_data[f'Predicted Sales for {category}'], label=f'Forecasted Sales - {category}', linestyle='--')

plt.title('Sales Category: Actual vs Forecasted')
plt.xlabel('Date')
plt.ylabel('Total Sales')

# Place the legend below the plot
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), shadow=True, ncol=2)

plt.grid(True)
plt.show()



### Save data to csv

In [ ]:
total_sales_df.to_csv('total_sales_df.csv')

In [ ]:
total_forecast.to_csv('total_forecast.csv')

In [ ]:
historical_sales_by_category_df.to_csv('historical_sales_by_category_df.csv')

In [ ]:
future_sales_predictions.to_csv('future_sales_predictions.csv')

In [ ]:
# Group by Sales
grouped_df = df.groupby(['OutDate', 'Sales']).agg({
    'OutCountTotal': 'sum',
    'TotalSales': 'sum',b
    'TotalSales_LocalPrice': 'sum'
}).reset_index()

# grouped_df.set_index('OutDate', inplace=True)
# grouped_df.sort_index(inplace=True)

grouped_df['OutDate'] = pd.to_datetime(grouped_df['OutDate'])
grouped_df.set_index('OutDate', inplace=True)

grouped_df.head()

In [ ]:
grouped_df.dtypes

In [ ]:
print("NaN values in the data:", df.isna().sum())

In [ ]:
# # Define a split date as a Timestamp object
# split_date = pd.Timestamp('2023-01-01')

# # Perform the split without setting OutDate as index
# train = grouped_df[grouped_df['OutDate'] < split_date]
# test = grouped_df[grouped_df['OutDate'] >= split_date]

# Splitting data into training and test sets
train = grouped_df[grouped_df.index.year <= 2022]  # Data up to the end of 2022
test = grouped_df[grouped_df.index.year == 2023]   # Data for 2023

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
from statsmodels.tsa.stattools import adfuller

def diagnose_model(train_data, test_data, column_to_forecast):
    # Fit the SARIMAX model
    order = (1, 1, 1)  # ARIMA model hyperparameters
    seasonal_order = (1, 1, 1, 12)  # Seasonal ARIMA model hyperparameters
    model = SARIMAX(train_data[column_to_forecast], order=order, seasonal_order=seasonal_order)
    results = model.fit()

    # Display the model summary
    print(results.summary())

    # Check for stationarity
    adf_test = adfuller(train_data[column_to_forecast])
    print(f'ADF Statistic: {adf_test[0]}')
    print(f'p-value: {adf_test[1]}')
    for key, value in adf_test[4].items():
        print('Critial Values:')
        print(f'   {key}, {value}')

    # Check if the forecast index has duplicates or irregularities
    forecast_index = pd.date_range(start=train_data.index[-1], periods=len(test_data) + 1, freq='D')[1:]
    print("\nDuplicates in the forecast index:", forecast_index.duplicated().any())
    print("Forecast index type:", forecast_index.dtype)

# Perform the diagnostic for each of the specified columns
for column in ['OutCountTotal', 'TotalSales', 'TotalSales_LocalPrice']:
    diagnose_model(train, test, column)


In [ ]:
def troubleshoot_forecast(train_data, test_data, column_to_forecast):
    # Fit the SARIMAX model
    order = (1, 1, 1)  # ARIMA model hyperparameters
    seasonal_order = (1, 1, 1, 12)  # Seasonal ARIMA model hyperparameters
    model = SARIMAX(train_data[column_to_forecast], order=order, seasonal_order=seasonal_order)
    results = model.fit()

    # Get the forecast
    forecast = results.get_forecast(steps=len(test_data))
    forecast_index = pd.date_range(start=train_data.index[-1], periods=len(test_data) + 1, freq='D')[1:]
    forecast_df = pd.DataFrame(forecast.predicted_mean, index=forecast_index, columns=[f'Forecast_{column_to_forecast}'])

    # Check for NaNs in forecast
    print(f"NaNs in forecast for {column_to_forecast}:")
    print(forecast_df.isna().sum())

    # Verify forecast index
    print(f"\nForecast index for {column_to_forecast}:")
    print(forecast_index)

    # Verify test data index
    print(f"\nTest data index for {column_to_forecast}:")
    print(test_data.index)

# Perform troubleshooting for each of the specified columns
for column in ['OutCountTotal', 'TotalSales', 'TotalSales_LocalPrice']:
    troubleshoot_forecast(train, test, column)


In [ ]:
def perform_forecast(train_data, test_data, column_to_forecast):
    # Fit the SARIMAX model
    order = (1, 1, 1)  # ARIMA model hyperparameters
    seasonal_order = (1, 1, 1, 12)  # Seasonal ARIMA model hyperparameters
    model = SARIMAX(train_data[column_to_forecast], order=order, seasonal_order=seasonal_order)
    results = model.fit()

    # Perform forecast
    forecast = results.get_forecast(steps=len(test_data))
    forecast_df = pd.DataFrame(forecast.predicted_mean, index=test_data.index, columns=[f'Forecast_{column_to_forecast}'])

    # Ensure test_data has the same index as forecast_df for proper alignment
    test_data = test_data.reindex(forecast_df.index)

    # Drop any rows with NaN values in either the test data or the forecast
    combined_df = pd.concat([test_data[column_to_forecast], forecast_df], axis=1).dropna()

    # If combined_df is empty, there's an issue with the data alignment
    if combined_df.empty:
        raise ValueError(f"No overlapping data to evaluate for column {column_to_forecast}")

    # Evaluate the forecast
    mae = mean_absolute_error(combined_df[column_to_forecast], combined_df.iloc[:, 1])
    print(f'Mean Absolute Error for {column_to_forecast}: {mae}')

    # Plot the forecast against the actual values
    plt.figure(figsize=(10, 5))
    train_data[column_to_forecast].plot(label='Train', legend=True)
    combined_df[column_to_forecast].plot(label='Test', legend=True, alpha=0.7)
    combined_df.iloc[:, 1].plot(label='Forecast', legend=True, color='red', alpha=0.7)

    plt.title(f'Forecast vs Actuals for {column_to_forecast}')
    plt.show()

In [ ]:
# Perform forecasting for each of the specified columns
for column in ['OutCountTotal', 'TotalSales', 'TotalSales_LocalPrice']:
    perform_forecast(train, test, column)

In [ ]:
def create_forecast_model(train_data, column_to_forecast, forecast_periods):
    # Fit the SARIMAX model
    order = (1, 1, 1)  # ARIMA model hyperparameters
    seasonal_order = (1, 1, 1, 12)  # Seasonal ARIMA model hyperparameters
    model = SARIMAX(train_data[column_to_forecast], order=order, seasonal_order=seasonal_order)
    results = model.fit()

    # Generate forecast
    forecast = results.get_forecast(steps=forecast_periods)
    forecast_index = pd.date_range(start=train_data.index[-1], periods=forecast_periods + 1, freq='D')[1:]
    forecast_df = pd.DataFrame(forecast.predicted_mean, index=forecast_index, columns=[f'Forecast_{column_to_forecast}'])

    return forecast_df

In [ ]:
# Forecasting for each column
forecast_periods = len(test)  # Number of periods in 2023 in your dataset
forecasts = {}
for column in ['OutCountTotal', 'TotalSales', 'TotalSales_LocalPrice']:
    forecasts[column] = create_forecast_model(train, column, forecast_periods)

# Accessing forecasted data
outcount_forecast = forecasts['OutCountTotal']
totalsales_forecast = forecasts['TotalSales']
totalsales_localprice_forecast = forecasts['TotalSales_LocalPrice']

print(outcount_forecast)